In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import hvplot.pandas

# Import API key
from api_keys import weather_api_key


In [3]:
df = pd.read_table('flight_edges.tsv', names=['Origin Airport', 'Destination Airport', 'Origin City', 'Destination City', 
                                                          'Passengers', 'Seats', 'Flights', 'Distance', 'Fly Date', 'Origin Population', 'Destination Population'])
df.head()

,Origin Airport,Destination Airport,Origin City,Destination City,Passengers,Seats,Flights,Distance,Fly Date,Origin Population,Destination Population
0,MHK,AMW,"Manhattan, KS","Ames, IA",21,30,1,254.0,200810,122049,86219
1,EUG,RDM,"Eugene, OR","Bend, OR",41,396,22,103.0,199011,284093,76034
2,EUG,RDM,"Eugene, OR","Bend, OR",88,342,19,103.0,199012,284093,76034
3,EUG,RDM,"Eugene, OR","Bend, OR",11,72,4,103.0,199010,284093,76034
4,MFR,RDM,"Medford, OR","Bend, OR",0,18,1,156.0,199002,147300,76034


In [4]:
# Determine the route by the origin airport and the destination airport

df["Route"] = df["Origin Airport"] + "_" + df["Destination Airport"]
passenger_count = df[["Route", "Passengers"]].groupby("Route").sum().sort_values("Passengers", ascending=False)

demand = passenger_count.merge(df[["Route", "Origin Airport", "Origin City", "Destination Airport", "Destination City"]], on="Route", how="inner").groupby("Route").first().sort_values("Passengers", ascending=False)

top_ten_routes_df = demand.head(10).copy()
top_ten_routes_df

,Passengers,Origin Airport,Origin City,Destination Airport,Destination City
Route,,,,,
OGG_HNL,32364612,OGG,"Kahului, HI",HNL,"Honolulu, HI"
HNL_OGG,29744742,HNL,"Honolulu, HI",OGG,"Kahului, HI"
LAX_HNL,28964154,LAX,"Los Angeles, CA",HNL,"Honolulu, HI"
HNL_LAX,28632161,HNL,"Honolulu, HI",LAX,"Los Angeles, CA"
LAS_LAX,26333721,LAS,"Las Vegas, NV",LAX,"Los Angeles, CA"
LAX_LAS,26177809,LAX,"Los Angeles, CA",LAS,"Las Vegas, NV"
LAX_SFO,25661782,LAX,"Los Angeles, CA",SFO,"San Francisco, CA"
SFO_LAX,25458207,SFO,"San Francisco, CA",LAX,"Los Angeles, CA"
ATL_MCO,23483751,ATL,"Atlanta, GA",MCO,"Orlando, FL"


In [5]:
# Set the API base URL

url = "http://api.openweathermap.org/data/2.5/weather?"

city_coordinates = {}

# Determine the latitude and longitude of the origin city and destination city

for city in top_ten_routes_df["Origin City"]:
    if city not in city_coordinates:
        city_coordinates[city] = {}
        city_url = f"{url}q={city[:-4]}&appid={weather_api_key}"

        try:
            city_data = requests.get(city_url).json()
            city_coordinates[city]["longitude"] = city_data["coord"]["lon"]
            city_coordinates[city]["latitude"] = city_data["coord"]["lat"]

        except:
            pass




for city in top_ten_routes_df["Destination City"]:
    if city not in city_coordinates:
        city_coordinates[city] = {}
        city_url = f"{url}q={city[:-4]}&appid={weather_api_key}"

        try:
            city_data = requests.get(city_url).json()
            city_coordinates[city]["longitude"] = city_data["coord"]["lon"]
            city_coordinates[city]["latitude"] = city_data["coord"]["lat"]

        except:
            pass

    

In [6]:
# Defined coordinates for latitude and longitude of the origin cities and destination cities

def get_coordinates(coord_type):
    def inner(city):
        return city_coordinates[city][coord_type]
    return inner


top_ten_routes_df["Origin Longitude"] = top_ten_routes_df["Origin City"].apply(get_coordinates("longitude"))
top_ten_routes_df["Origin Latitude"] = top_ten_routes_df["Origin City"].apply(get_coordinates("latitude"))

top_ten_routes_df["Destination Longitude"] = top_ten_routes_df["Destination City"].apply(get_coordinates("longitude"))
top_ten_routes_df["Destination Latitude"] = top_ten_routes_df["Destination City"].apply(get_coordinates("latitude"))

top_ten_routes_df


,Passengers,Origin Airport,Origin City,Destination Airport,Destination City,Origin Longitude,Origin Latitude,Destination Longitude,Destination Latitude
Route,,,,,,,,,
OGG_HNL,32364612,OGG,"Kahului, HI",HNL,"Honolulu, HI",-156.4700,20.8947,-157.8583,21.3069
HNL_OGG,29744742,HNL,"Honolulu, HI",OGG,"Kahului, HI",-157.8583,21.3069,-156.4700,20.8947
LAX_HNL,28964154,LAX,"Los Angeles, CA",HNL,"Honolulu, HI",-118.2437,34.0522,-157.8583,21.3069
HNL_LAX,28632161,HNL,"Honolulu, HI",LAX,"Los Angeles, CA",-157.8583,21.3069,-118.2437,34.0522
LAS_LAX,26333721,LAS,"Las Vegas, NV",LAX,"Los Angeles, CA",-115.1372,36.1750,-118.2437,34.0522
LAX_LAS,26177809,LAX,"Los Angeles, CA",LAS,"Las Vegas, NV",-118.2437,34.0522,-115.1372,36.1750
LAX_SFO,25661782,LAX,"Los Angeles, CA",SFO,"San Francisco, CA",-118.2437,34.0522,-122.4194,37.7749
SFO_LAX,25458207,SFO,"San Francisco, CA",LAX,"Los Angeles, CA",-122.4194,37.7749,-118.2437,34.0522
ATL_MCO,23483751,ATL,"Atlanta, GA",MCO,"Orlando, FL",-84.3880,33.7490,-81.3792,28.5383


In [10]:
# Plot the origin and destination cities

origin_plot = top_ten_routes_df.hvplot.points(
    "Origin Longitude",
    "Origin Latitude",
    geo = True,
    tiles = "OSM",
    frame_width = 800, 
    frame_height = 600,
    scale = 1,
    size = 100,
    marker = "^",
    color = "Origin City",
    title = "Top Seven Originating Cities",
    hover_cols = ["Origin City", "Origin Airport", "Destination City", "Destination Airport"]    
)
origin_plot.opts(fontsize={'title':16},title="Top Seven Originating Cities")
# destination_plot = top_ten_routes_df.hvplot.points(
#     "Destination Longitude",
#     "Destination Latitude",
#     geo = True,
#     tiles = "OSM",
#     frame_width = 800, 
#     frame_height = 600,
#     scale = 1,
#     size = 100,
#     marker = "v",
#     color = "Destination City",
#     hover_cols = ["Origin City", "Origin Airport", "Destination City", "Destination Airport"]
# )

# city_plot = origin_plot * destination_plot
# city_plot
origin_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Origin Longitude,Origin Latitude]   (Origin City,Origin Airport,Destination City,Destination Airport)

In [11]:
destination_plot = top_ten_routes_df.hvplot.points(
    "Destination Longitude",
    "Destination Latitude",
    geo = True,
    tiles = "OSM",
    frame_width = 800, 
    frame_height = 600,
    scale = 1,
    size = 100,
    marker = "o",
    color = "Destination City",
    title = "Top Six Destination Cities",
    hover_cols = ["Origin City", "Origin Airport", "Destination City", "Destination Airport"]
)

destination_plot.opts(fontsize={'title':16},title="Top Six Destination Cities")
destination_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Destination Longitude,Destination Latitude]   (Destination City,Origin City,Origin Airport,Destination Airport)